In [2]:
# imports
# type: ignore
from __future__ import annotations
from datetime import datetime as dt
import cProfile
from pprint import pprint
from typing import Any

import aiohttp

from vindicator import *
from tests.mock_wynnapi import MockWynnApi

In [12]:
def className(obj: object) -> str:
    return obj.__class__.__name__

def getIndent(depth: int) -> str:
    return '    ' * depth

def printDictTypes(dict_: dict[str, Any], depth: int = 0) -> None:
    for key, value in dict_.items():
        if isinstance(value, dict):
            print(f'{getIndent(depth)}{key}: dict')
            printDictTypes(value, depth + 1)
        else:
            print(f'{getIndent(depth)}{key}: {className(value)}')

async def get_full_player_stats(username_or_uuid: str) -> tuple[dict[str, Any], dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get(f"https://api.wynncraft.com/v3/player/{username_or_uuid}?fullResult") as resp:
            return (await resp.json(), dict(resp.headers))

async def get_guild_stats(guildname_or_prefix: str, is_prefix: bool = False) -> tuple[dict[str, Any], dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get(f"https://api.wynncraft.com/v3/guild/{'prefix/' if is_prefix else ''}{guildname_or_prefix}") as resp:
            return (await resp.json(), dict(resp.headers))

async def get_online_players() -> tuple[dict[str, Any], dict[str, Any]]:
    async with aiohttp.ClientSession() as session:
        async with session.get("https://api.wynncraft.com/v3/player?identifier=uuid") as resp:
            return (await resp.json(), dict(resp.headers))

# Sandbox

In [13]:
repo = WynnDataRepository()
mockdata = MockWynnApi()

p: PlayersResponse = mockdata.onlineuuids  # type: ignore
ps: list[PlayerResponse] = mockdata.onlineplayerstats  # type: ignore
gs: list[GuildResponse] = mockdata.onlineguildstats  # type: ignore

In [14]:
await repo.character_history_repository.create_table()
await repo.character_info_repository.create_table()
await repo.guild_history_repository.create_table()
await repo.guild_info_repository.create_table()
await repo.guild_member_history_repository.create_table()
await repo.online_players_repository.create_table()
await repo.player_activity_history_repository.create_table()
await repo.player_history_repository.create_table()
await repo.player_info_repository.create_table()

In [15]:
print(await repo.character_history_repository.insert(CharacterHistory.from_responses(ps)))
print(await repo.character_info_repository.insert(CharacterInfo.from_responses(ps)))
print(await repo.guild_history_repository.insert(GuildHistory.from_responses(gs)))
print(await repo.guild_info_repository.insert(GuildInfo.from_responses(gs)))
print(await repo.guild_member_history_repository.insert(GuildMemberHistory.from_responses(gs)))
print(await repo.online_players_repository.insert(OnlinePlayers.from_response(p)))
# print(await repo.player_activity_history_repository.insert())
print(await repo.player_history_repository.insert(PlayerHistory.from_responses(ps)))
print(await repo.player_info_repository.insert(PlayerInfo.from_responses(ps)))

2925
0
114
114
249
-1
657
657


In [16]:
print(await repo.character_history_repository.count())
print(await repo.character_info_repository.count())
print(await repo.guild_history_repository.count())
print(await repo.guild_info_repository.count())
print(await repo.guild_member_history_repository.count())
print(await repo.online_players_repository.count())
# print(await repo.player_activity_history_repository.count())
print(await repo.player_history_repository.count())
print(await repo.player_info_repository.count())

2925
2925
114
114
249
657
657
657


# Archive

In [17]:
# list of objects: `in`, `min()`, `max()`
# class Foo:
#     def __init__(self, weight: int, value: str):
#         self.weight = weight
#         self.value = value

#     def __eq__(self, value: object) -> bool:
#         return self.value == value

#     def __lt__(self, other: Foo) -> bool:
#         return self.weight < other.weight

# ch = 'abcdefghijklmnopqrstuvwxyz'
# foos = []
# for i in range(26):
#     foos.append(Foo(i, ch[i]))

# # 'value' in 'iterable' checks for equality of 'value' with each element in 'iterable'
# print('g' in foos)
# # min and max use the __lt__ magic method to compare elements in 'iterable'
# print(min(foos).value)
# print(max(foos).value)

In [18]:
# dbmodel creation algorithm perfomance compariosn
# from tests.mock_wynnapi import MockWynnApi
# from vindicator.api.wynn.player_response import PlayerResponse
# from vindicator.db.wynndata.model.character_history.character_history import CharacterHistory
# from vindicator.db.wynndata.model.character_info.character_info import CharacterInfo
# from vindicator.db.wynndata.model.player_history.player_history import PlayerHistory
# from vindicator.db.wynndata.model.player_info.player_info import PlayerInfo

# mockwynnapi = MockWynnApi()
# data: list[PlayerResponse] = mockwynnapi.onlineplayerstats  # type: ignore

# def withloop():
#     playerinfo = []
#     characterinfo = []
#     playerhistory = []
#     characterhistory = []
#     for resp in data:
#         playerinfo.extend(PlayerInfo.from_responses((resp,)))
#         characterinfo.extend(CharacterInfo.from_responses((resp,)))
#         playerhistory.extend(PlayerHistory.from_responses((resp,)))
#         characterhistory.extend(CharacterHistory.from_responses((resp,)))

# def without():
#     playerinfo: tuple[PlayerInfo, ...] = PlayerInfo.from_responses(data)
#     characterinfo: tuple[CharacterInfo, ...] = CharacterInfo.from_responses(data)
#     playerhistory: tuple[PlayerHistory, ...] = PlayerHistory.from_responses(data)
#     characterhistory: tuple[CharacterHistory, ...] = CharacterHistory.from_responses(data)

# cProfile.run('withloop()', sort='time')
# cProfile.run('without()', sort='time')

In [19]:
# += vs extend()
# r = range(50_000_000)
# def add():
#     l = []
#     l += tuple(r)
# def extend():
#     l = []
#     l.extend(r)

# cProfile.run('add()')
# cProfile.run('extend()')

In [20]:
# moto-bot repository structure
# from __future__ import annotations
# ID = TypeVar('ID', contravariant=True)
# T = TypeVar('T')
# S = TypeVar('S')
# U = TypeVar('U', bound=type)

# def interface(t: U) -> U: return t
# def abstract(t: U) -> U: return t

# @interface
# class Table(Generic[T, ID], Protocol):
#     def create(self, entity: T) -> bool: ...
#     def exists(self, id_: ID) -> bool: ...
#     def count(self) -> float: ...
#     def findOne(self, id_: ID) -> None | T: ...
#     def findAll(self) -> None | list[T]: ...
#     def update(self, entity: T) -> bool: ...
#     def delete(self, id_: ID) -> bool: ...

# @abstract
# class MariaRepository(ABC, Generic[S]):
#     def __init__(self, db: Any, logger: Any) -> None:
#         self._db: Any = db
#         self._logger: Any = logger
#     def _prepareStatement(self, connection: Any, sql: str, objects: list[object]) -> Any: ...
#     def _execute(self, sql: str, strings: list[object]) -> bool: ...
#     # def _execute(self, connection: Any, sql: str, objects: list[object]) -> bool: ...
#     def _executeQuery(self, sql: str, objects: list[object]) -> None | Any: ...
#     def _logResponseException(self, e: Any) -> None: ...
#     def _bindAll(self, res: Any) -> list[S]: ...
#     @abstractmethod
#     def _bind(self, res: Any) -> S: ...

# @interface
# class GuildId(Protocol):
#     def getName(self) -> str: ...

# class Guild(GuildId):
#     def __init__(self, name: str, prefix: str, createdAt: dt) -> None:
#         self._name: str = name
#         self._prefix: str = prefix
#         self._createdAt: dt = createdAt
#     @override
#     def getName(self) -> str: return self._name
#     def getPrefix(self) -> str: return self._prefix
#     def getCreatedAt(self) -> dt: return self._createdAt

# @interface
# class CharacterHistoryBase(Table[Guild, GuildId], Protocol):
#     def findAllIn(self, guildNames: list[str]) -> None | list[Guild]: ...
#     def findAllCaseInsensitive(self, guildName: str) -> None | list[Guild]: ...
#     def findAllByPrefix(self, prefix: str) -> None | list[Guild]: ...
#     def findAllByPrefixCaseInsensitive(self, prefix: str) -> None | list[Guild]: ...

# class CharacterHistoryTable(MariaRepository[Guild], CharacterHistoryBase):
#     """class::MariaGuildRepository"""
#     _DB_FORMAT: Any
#     def __init__(self, db: Any, logger: Any) -> None:
#         super().__init__(db, logger)
#     @override
#     def _bind(self, res: Any) -> Guild: ...
#     @override
#     def create(self, entity: Guild) -> bool: ...
#     @override
#     def exists(self, id_: GuildId) -> bool: ...
#     @override
#     def count(self) -> float: ...
#     @override
#     def findOne(self, id_: GuildId) -> None | Guild: ...
#     @override
#     def findAllIn(self, guildNames: list[str]) -> None | list[Guild]: ...
#     def findAllCaseInsensitive(self, guildName: str) -> None | list[Guild]: ...
#     def findAllByPrefix(self, prefix: str) -> None | list[Guild]: ...
#     def findAllByPrefixCaseInsensitive(self, prefix: str) -> None | list[Guild]: ...
#     @override
#     def findAll(self) -> None | list[Guild]: ...
#     @override
#     def update(self, entity: Guild) -> bool: ...
#     @override
#     def delete(self, id_: GuildId) -> bool: ...
